<a href="https://colab.research.google.com/github/RodgelineBansaganBSIT2B/Bansagan-Rodgeline-and-Simbanagan-Jonnerey-Activity-1-/blob/main/LabActivity3_1_BANSAGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


*A flight data warehouse for a travel agent consists of six dimensions: *
*traveller, departure (city), departure time, arrival, arrival time, and flight; and two measures count, and avg fare where avg fare stores the concrete fare at the lowest level but average fare at other levels.*

**(a) Suppose the cube is fully materialized. Starting with the base cuboid [traveller, departure, departure time, arrival, arrival time, flight], what specific OLAP operations (e.g. roll-up flight to airline) should one perform in order to list the average fare per month for each business traveller who flies Philippine Airlines (PA) from Davao City to Manila in the year 2023?**

In [2]:
import pandas as pd

# Updated data with 10 travelers and fares above 5000
data = {
    'traveller': ['Jack', 'Olivia', 'Charlie', 'Olivia', 'Jack', 'Sophia', 'Daniel', 'Emily', 'Matthew', 'Lucy'],
    'departure_city': ['Davao', 'Davao', 'Davao', 'Davao', 'Davao', 'Davao', 'Davao', 'Davao', 'Davao', 'Davao'],
    'departure_time': ['2023-01-15', '2023-02-20', '2023-03-10', '2023-04-05', '2023-05-12', '2023-06-18', '2023-07-25', '2023-08-30', '2023-09-10', '2023-10-05'],
    'arrival_city': ['Manila', 'Manila', 'Manila', 'Manila', 'Manila', 'Manila', 'Manila', 'Manila', 'Manila', 'Manila'],
    'arrival_time': ['2023-01-16', '2023-02-21', '2023-03-11', '2023-04-06', '2023-05-13', '2023-06-19', '2023-07-26', '2023-08-31', '2023-09-11', '2023-10-06'],
    'flight': ['PA101', 'PA102', 'PA103', 'PB201', 'PA104', 'PA105', 'PA106', 'PA107', 'PA108', 'PA109'],
    'count': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    'fare': [5200, 5100, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000]
}

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Step 1: Filter the data
filtered_df = df[(df['departure_city'] == 'Davao') &
                 (df['arrival_city'] == 'Manila') &
                 (df['flight'].str.startswith('PA')) &
                 (df['departure_time'].str.startswith('2023'))]

# Step 2: Optionally roll-up (assuming flight dimension is already rolled up)
# Step 3: Drill-down to month level
filtered_df['departure_month'] = pd.to_datetime(filtered_df['departure_time']).dt.to_period('M')

# Step 4: Group-by traveller dimension
grouped_df = filtered_df.groupby(['traveller', 'departure_month'])

# Step 5: Aggregate to calculate average fare
result = grouped_df['fare'].mean().reset_index()

print(result)


  traveller departure_month    fare
0   Charlie         2023-03  5300.0
1    Daniel         2023-07  5700.0
2     Emily         2023-08  5800.0
3      Jack         2023-01  5200.0
4      Jack         2023-05  5500.0
5      Lucy         2023-10  6000.0
6   Matthew         2023-09  5900.0
7    Olivia         2023-02  5100.0
8    Sophia         2023-06  5600.0


<ipython-input-2-4ceed6e2cf9c>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['departure_month'] = pd.to_datetime(filtered_df['departure_time']).dt.to_period('M')


**(b) Suppose we want to compute a data cube where the conditions is that the minimum number of records is 10 and the average fare is over Php 5,000.00. Outline an efficient cube computation method (based on common sense about flight data distribution).**

In [3]:
# Step 1: Filtering
filtered_data = df[df['fare'] > 5000]

# Step 2: Grouping
grouped_data = filtered_data.groupby(['departure_city', 'arrival_city'])

# Step 3: Aggregation
aggregated_data = grouped_data.agg({'count': 'sum', 'fare': 'mean'}).reset_index()

# Step 4: Pruning
pruned_data = aggregated_data[aggregated_data['count'] >= 10]

# Step 5: Cube Formation
cube = pruned_data.pivot_table(index='departure_city', columns='arrival_city', values='fare')

print("Cube:")
print(cube)

Cube:
arrival_city    Manila
departure_city        
Davao           5550.0
